In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt

In [2]:
# import the data
data = pd.read_csv("clean_data_Alakrot.csv")
print (data.head())

   Unnamed: 0                               ORIGINAL_commentText  \
0           0  : O كاظم يسواكم بس انتم لان مقاماته تدرس فى ال...   
1           1                                     : سخط على ليكي   
2           2  ?  .  . مسائكم سعيد يا شباب , ممكن حد يقول لى ...   
3           3  ؟  ؟  ؟  ؟  ؟  ؟  فهمونا بس هو انشهر بلبنان ول...   
4           4   . \nانا ضد تصرفات احلام بس ليه اسب واشتم وفى ...   

                                         commentText Label  
0  : O كاظم يسواكم بس انتم لان مقاماته تدرس فى ال...     N  
1                                     : سخط على ليكي     P  
2  ?  .  . مسائكم سعيد يا شباب , ممكن حد يقول لى ...     N  
3  ؟  ؟  ؟  ؟  ؟  ؟  فهمونا بس هو انشهر بلبنان ول...     N  
4   .  انا ضد تصرفات احلام بس ليه اسب واشتم وفى ا...     N  


In [3]:
# get needed columns:
label_data = data[['commentText', 'Label']]
label_data[2000:2010]
# sample = label_data[:10000]

,commentText,Label
2000,الله يحسن حال العرب وبس 😊💔,N
2001,الله يحشرك مع بشار,P
2002,الله يحشرك مع بشار .,P
2003,الله يحفظك يااستاذكاظم الحب ياروووعه ياظاغطهم ...,N
2004,الله يحمى قيصر الغناء وهدا البقره يلى بدو يدخ...,P
2005,الله يحميك سوريا الاسد يا رب العالمين,N
2006,الله يحميك ويعطيكى طوله العمر انتى واولاد الب...,N
2007,الله يحميكى يا رغده و يحمى بشار الاسد الى حمى...,N
2008,الله يخارجنا كيف دى بيتحملها زوجها والله ودف م...,P
2009,الله يخارجنا منك يارب انت واخوكى الشاذ يا عبده...,P


In [37]:
# def clean_text(x):
#     return str(x)

# label_data["commentText"] = label_data["commentText"].apply(lambda x: clean_text(x))

In [6]:
# get the training model:
from sklearn.model_selection import train_test_split
# split data into (80% training and 20% testing):
train, test = train_test_split(label_data, test_size=0.2)# sample

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf feature to get the target words more accurate
from sklearn.model_selection import GridSearchCV # grid search feature to enhance trianing
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB # no needed here

from sklearn import svm
from sklearn.svm import SVC, LinearSVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
from sklearn.linear_model import SGDClassifier

# for evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score
from time import time

max_features = 5000 # to train the first 5000 instances as a standard for the whole corpus

x = train['commentText']
y = train['Label']


tfidf = TfidfVectorizer(max_features=max_features)

pipeline = Pipeline([
    ('tfidf', tfidf),
    ('SVM', svm.SVC(kernel='linear')),
])  # other parameters:
    # svm.SVC(kernel='linear')
    # SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
    # SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf', max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)
    # svm.SVC(kernel='rbf')
    # For more information, visit: https://scikit-learn.org/stable/modules/svm.html.

parameters = {
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2')   
}

grid_search = GridSearchCV(pipeline, parameters, cv=5,
                               n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
print(parameters)
t0 = time()
grid_search.fit(x, y)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
#refitting on entire training data using best settings
grid_search.refit

x_test = test['commentText']
y_test = test['Label']

predicted = grid_search.predict(x_test)
print("accuracy=", np.mean(predicted == y_test))
print("recall=", recall_score(y_test, predicted, average='weighted') )
print("precision=", precision_score(y_test, predicted, average='weighted'))
print("weighted f-score", f1_score(y_test, predicted, average='weighted'))
print("Confusion matrix\n", confusion_matrix(y_test, predicted))

Performing grid search...
pipeline: ['tfidf', 'SVM']
parameters:
{'tfidf__use_idf': (True, False), 'tfidf__norm': ('l1', 'l2')}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   49.0s finished


done in 53.926s

Best score: 0.788
Best parameters set:
	tfidf__norm: 'l2'
	tfidf__use_idf: True
accuracy= 0.7889581478183437
recall= 0.7889581478183437
precision= 0.7989479177577123
weighted f-score 0.7833096208783122
Confusion matrix
 [[1142  112]
 [ 362  630]]
